# Deployment
In this notebook, our objective is to deploy Hugging Face Model of Bert_qa

# Notebook Overview
- Install requirements and Imports Dependencies
- Configurations
- Model Load
- Model Registry
- Testing latest model registred

# Install requirements and Imports Dependencies 

In [ ]:
!pip install -r ../requirements.txt --quiet

In [ ]:
# Standard Library Imports
import logging
import warnings

# Third-Party Libraries
import shutil

# MLflow for Experiment Tracking and Model Management
import mlflow
from mlflow.types.schema import Schema, ColSpec
from mlflow.types import ParamSchema, ParamSpec
from mlflow.models import ModelSignature

# Transformers
from transformers import pipeline

# Configurations

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
# Define global experiment and run names to be used throughout the notebook
MODEL_PERSONAL_NAME = "morgana-rodrigues/bert_qa"
EXPERIMENT_NAME = "BERT model for Q&A"
MODEL_NAME = "BERT_QA"

# Set up the paths
MODEL_PATH = "../models:/BERT_QA"

# Set up the chunk separator for text processing
CHUNK_SEPARATOR = "\n\n"

In [ ]:
# === Create logger ===
logger = logging.getLogger("deployment-notebook")
logger.setLevel(logging.INFO)

formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s", 
                             datefmt="%Y-%m-%d %H:%M:%S") 

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.propagate = False

In [ ]:
logger.info('Notebook execution started.')

## Model

In this part of the code, we load a Transformer model saved on Hugging Face to use it locally (in a pipeline object). This pipeline is then tested with a simple sample.

In [ ]:
model_name = MODEL_PERSONAL_NAME

qa_pipeline = pipeline(
    'question-answering',
    model=model_name,
    device=0 # -1 means running on CPU
)

In [ ]:
qa_pipeline (context="Take me down to Paradise City where the grass is green and the girls are pretty", question="What colour is the grass")

This class below encapsulates the model in the format that will be logged/registered into MLFlow. It receives a pipeline (or a trainer) as input, saves the model into a temporary folder (called model_name), and log as an artifact into MLFlow. When MLFlow deploys the model, it loads these artifacts into a new pipeline, which can be used to perform inference.

In [ ]:
class DistilBERTModel(mlflow.pyfunc.PythonModel):
    def _preprocess(self, inputs):
        """
        Preprocesses the input data.

        Args:
            inputs: A dictionary containing two keys:
                - 'context': A list with the context text.
                - 'question': A list with the question to be answered.

        Returns:
            tuple: A tuple containing the context (str) and the question (str).
        """
        try:
            context = inputs['context'][0]
            question = inputs['question'][0]
            logger.info("Preprocessing input:", context, question)
            return context, question
        
        except Exception as e:
            logger.error(f"Error preprocessing the input data: {str(e)}")  

    def load_context(self, context):
        """
        Loads the question-answering pipeline using the saved model artifact.

        Args:
            context: The MLflow context object 
                containing the loaded artifacts.
        """
        try:
            self.model = pipeline(
                'question-answering',
                model=context.artifacts["model"],
                device=0
            )
        except Exception as e:
            logger.error(f"Error loading the question-answering pipeline: {str(e)}")     

    def predict(self, context, model_input):
        """
        Runs inference using the loaded model and input data.

        Args:
            context: The MLflow context object 
                with access to artifacts.
            model_input: A dictionary containing 'context' and 'question' keys.

        Returns:
            The output from the model containing the predicted answer and optionally the score.
        """
        try:
            in_ctx, question = self._preprocess(model_input)
            output = self.model(context=in_ctx, question=question)
            return output
        except Exception as e:
            logger.error(f"Error running inference: {str(e)}")  

    @classmethod
    def log_model(cls, model_name, source_trainer=None, source_pipeline=None, demo_folder="demo"):
        """
        Logs the model to MLflow, including artifacts, dependencies, and input/output signatures.

        Args:
            model_name: Path where the model will be temporarily saved before logging.
            source_trainer: A trainer object with a `.save_model()` method. Defaults to None.
            source_pipeline: A pipeline object with a `.save_pretrained()` method. Defaults to None.
            demo_folder: Path to the folder containing the compiled demo UI. Defaults to "demo".
        """
        try:
            input_schema = Schema([
                ColSpec("string", "context"),
                ColSpec("string", "question"),
            ])

            output_schema = Schema([
                ColSpec("string", "answer")
            ])

            params_schema = ParamSchema([
                ParamSpec("show_score", "boolean", False)
            ])

            signature = ModelSignature(
                inputs=input_schema,
                outputs=output_schema,
                params=params_schema
            )

            # Save the model locally from the trainer or pipeline
            if source_trainer is not None:
                source_trainer.save_model(model_name)
            elif source_pipeline is not None:
                source_pipeline.save_pretrained(model_name)

            # List of dependencies to include in the environment
            requirements = [
                "transformers==4.47.0",
                "tf_keras"
            ]

            # Log the model to MLflow
            mlflow.pyfunc.log_model(
                model_name,
                python_model=cls(),
                artifacts={
                    "model": model_name,
                    "demo": demo_folder
                },
                signature=signature,
                pip_requirements=requirements
            )

            # Remove the temporary model folder
            shutil.rmtree(model_name)
            logger.info("Logging model to MLflow done successfully")

        except Exception as e:
            logger.error(f"Error logging model to MLflow: {str(e)}")

# Model Registry

In [ ]:
mlflow.set_experiment(experiment_name = EXPERIMENT_NAME)

In [ ]:
with mlflow.start_run(run_name= MODEL_NAME) as run:
    logger.info(f"Run's Artifact URI: {run.info.artifact_uri}")
    DistilBERTModel.log_model(model_name = MODEL_NAME, source_pipeline=qa_pipeline)
    mlflow.register_model(model_uri = f"runs:/{run.info.run_id}/{MODEL_NAME}", name = MODEL_NAME)
    logger.info("Model registred!")

# Testing latest model registred

In [ ]:
client = mlflow.MlflowClient()
model_metadata = client.get_latest_versions(MODEL_NAME, stages=["None"])
latest_model_version = model_metadata[0].version
logger.info(latest_model_version, mlflow.models.get_model_info(f"{MODEL_PATH}/{latest_model_version}").signature)

In [ ]:
model = mlflow.pyfunc.load_model(model_uri=f"{MODEL_PATH}/{latest_model_version}")
context = "Marta is mother of John and Amanda"
question = "what is the name of Marta's daugther?"
model.predict({"context": [context], "question":[question]})

Built with ❤️ using Z by HP AI Studio.